In [17]:
import sys
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, LSTM
from keras.layers.embeddings import Embedding

In [18]:
print ('hello word')

hello word


In [3]:
raw_text = open('wonderland.txt').read()
raw_text = raw_text.lower()
print (raw_text[:200])

alice's adventures in wonderland

lewis carroll

the millennium fulcrum edition 3.0

chapter i. down the rabbit-hole

alice was beginning to get very tired of sitting by her sister on the
bank, and of


In [4]:
# transform language into numbers
chars = sorted(list(set(raw_text)))
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters:", n_chars)
print ("Total Vocab:", n_vocab)
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

Total Characters: 144413
Total Vocab: 45


In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/cruiser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
from nltk import tokenize
sentences = tokenize.sent_tokenize(raw_text)
lengths = []
for i in range(len(sentences)):
    lengths.append(len(sentences[i]))
lengths = np.array(lengths)
sentences[0]

"alice's adventures in wonderland\n\nlewis carroll\n\nthe millennium fulcrum edition 3.0\n\nchapter i. down the rabbit-hole\n\nalice was beginning to get very tired of sitting by her sister on the\nbank, and of having nothing to do: once or twice she had peeped into the\nbook her sister was reading, but it had no pictures or conversations in\nit, 'and what is the use of a book,' thought alice 'without pictures or\nconversations?'"

In [7]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length):
    seq_in  = raw_text[i: i+seq_length]
    seq_out = raw_text[i+seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ('Total patterns:', n_patterns)

Total patterns: 144313


# Small LSTM Network : Constant Characters

In [8]:
# Reshape dataX to size of [samples, time steps, features] and scale it to 0-1
# Represent dataY as one hot encoding
X_train = np.reshape(dataX, (n_patterns, seq_length, 1))/float(n_vocab)
Y_train = np_utils.to_categorical(dataY)

In [9]:
train=(X_train*(float(n_vocab))).astype(int)

In [10]:
n_vocab

45

In [11]:
def read_out(X_train,n_vocab):
    chars=[]
    train=(X_train*(float(n_vocab))).astype(int)
    for sentence in train:
        chars.append( "".join([int_to_char[index[0]] for index in sentence]))
    return chars

In [12]:
chars=read_out (X_train,n_vocab)
chars[0]

"alice's adventures in wonderland\n\nlewis carroll\n\ntge millennium fulcrum edition 3.0\n\ncgapter i. down"

In [ ]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0

In [13]:
model = Sequential()
model.add(LSTM(256, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(Y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               264192    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 45)                11565     
Total params: 275,757
Trainable params: 275,757
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [33]:
model.fit(X_train[:1000], Y_train[:1000], nb_epoch=10, batch_size=64, callbacks=callbacks_list)

/home/cruiser/anaconda3/lib/python3.6/site-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


RuntimeError: The model needs to be compiled before being used.

In [31]:
word=model.predict(X_train[3:6])
print (type(word[0][]))

<class 'numpy.float32'>


In [33]:
def read_out(X_train,n_vocab):
    chars=[]
    train=(X_train*(float(n_vocab))).astype(int)
    for sentence in train:
        chars.append( "".join([int_to_char[index] for index in sentence]))
    return chars
chars=read_out (word,n_vocab)
chars

['    \n\n\n\n\n\n \n\n\n\n\n\n\n\n  \n       \n\n\n\n   \n   \n\n\n \n',
 '    \n\n\n\n\n\n \n\n\n\n\n\n\n\n  \n       \n\n\n\n   \n   \n\n\n \n',
 '   \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n  \n       \n\n\n\n   \n   \n\n\n \n']

In [34]:
# load the network weights
filename = "weights-improvement-16-2.0185.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

OSError: Unable to open file (Unable to open file: name = 'weights-improvement-16-2.0185.hdf5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

In [14]:
start = np.random.randint(0, len(X_train)-1)
pattern = dataX[start]
print "Seed:"
print "\"", ''.join([int_to_char[value] for value in pattern]), "\""
print "\nGenerated Sequence:"
for i in xrange(200):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x/float(n_vocab)
    prediction = model.predict(x)
    index = np.argmax(prediction)
    result = int_to_char[index]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print "\nDone."    

Seed:
" e
to them, and all dripping wet, cross, and uncomfortable.

the first question of course was, how to "
 soenk at the could ao a lore of the sabbit sar and aalen and all the doere sf teered the had beee oo the daneer  and whet soeee to toink at ierse the saabit sareen the was aol a crea and an a lore of
Done.


# Small LSTM Network : Constant Characters With Embedding Layers

In [7]:
X_train = np.reshape(dataX, (n_patterns, seq_length))/float(n_vocab)
Y_train = np_utils.to_categorical(dataY)

NameError: name 'dataX' is not defined

In [8]:
embedding_vector_length = 32
model1 = Sequential()
model1.add(Embedding(n_vocab, embedding_vector_length, input_length=seq_length))
model1.add(LSTM(256))
model1.add(Dropout(0.2))
model1.add(Dense(Y_train.shape[1], activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam')
print (model1.summary())

NameError: name 'n_vocab' is not defined

In [ ]:
model1.fit(X_train, Y_train, nb_epoch=10, batch_size=512)

/home/cruiser/anaconda3/lib/python3.6/site-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/10


In [1]:
start = np.random.randint(0, len(X_train)-1)
pattern = dataX[start]
print "Seed:"
print "\"", ''.join([int_to_char[value] for value in pattern]), "\""
for i in xrange(500):
    x = np.reshape(pattern, (1, len(pattern)))
    x = x/float(n_vocab)
    prediction = model1.predict(x)
    index = np.argmax(prediction)
    result = int_to_char[index]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nDone.")    

# Deep LSTM Network : Constant Characters

In [32]:
# Reshape dataX to size of [samples, time steps, features] and scale it to 0-1
# Represent dataY as one hot encoding
X_train = np.reshape(dataX, (n_patterns, seq_length, 1))/float(n_vocab)
Y_train = np_utils.to_categorical(dataY)

In [51]:
model = Sequential()
model.add(LSTM(256, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(Y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
print model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_14 (LSTM)                   (None, 100L, 256)     264192      lstm_input_8[0][0]               
____________________________________________________________________________________________________
dropout_11 (Dropout)             (None, 100L, 256)     0           lstm_14[0][0]                    
____________________________________________________________________________________________________
lstm_15 (LSTM)                   (None, 64)            82176       dropout_11[0][0]                 
____________________________________________________________________________________________________
dropout_12 (Dropout)             (None, 64)            0           lstm_15[0][0]                    
___________________________________________________________________________________________

In [53]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [2]:
model.fit(X_train, Y_train, nb_epoch=2, batch_size=256, callbacks=callbacks_list)